<a href="https://colab.research.google.com/github/Alexis20723/computo-en-la-nube/blob/main/Comandos_archivo_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Abrir archivos**

In [ ]:
fh = open('fear.txt', 'rt') # r: read, t: text
for line in fh.readlines():
  print(line.strip())
fh.close()

# **Leer y escribir en un archivo**

In [ ]:
with open('print_example.txt', 'w') as fw:
  print('HEscribiendo en un archivo!!!', file=fw)

# **Comprobar existencia de un archivo**

In [3]:
from pathlib import Path
p = Path('fear.txt')
path = p.parent.absolute()
print(p.is_file()) # True
print(path) # /Users/fab/srv/lpp3e/ch08/files
print(path.is_dir()) # True
q = Path('/Users/fab/srv/lpp3e/ch08/files')
print(q.is_dir()) # True

False
/content
True
False


# **Manipular nombres**

In [ ]:
from pathlib import Path
p = Path('fear.txt')
print(p.absolute())
print(p.name)
print(p.parent.absolute())
print(p.suffix)
print(p.parts)
print(p.absolute().parts)
readme_path = p.parent / '..' / '..' / 'README.rst'
print(readme_path.absolute())
print(readme_path.resolve())

# **Comprimir archivos y directorios**

In [ ]:
from zipfile import ZipFile
with ZipFile('example.zip', 'w') as zp:
  zp.write('content1.txt')
  zp.write('content2.txt')
  zp.write('subfolder/content3.txt')
  zp.write('subfolder/content4.txt')
with ZipFile('example.zip') as zp:
  zp.extract('content1.txt', 'extract_zip')
  zp.extract('subfolder/content3.txt', 'extract_zip')

# **Trabajar con archvivos .JSON**

In [ ]:
import sys
import json
data = {
  'big_number': 2 ** 3141,
  'max_float': sys.float_info.max,
  'a_list': [2, 3, 5, 7],
}
json_data = json.dumps(data)
data_out = json.loads(json_data)
assert data == data_out

# **Flujos de memoria**

In [ ]:
import io
stream = io.StringIO()
stream.write('Aprendiendo a programar en Python.\n')
print('Muy bien!', file=stream)
contents = stream.getvalue()
print(contents)
stream.close()

# **Solicitudes de HTTP**

In [ ]:
import requests
urls = {
  "get": "https://httpbin.org/get?t=learn+python+programming",
  "headers": "https://httpbin.org/headers",
  "ip": "https://httpbin.org/ip",
  "user-agent": "https://httpbin.org/user-agent",
  "UUID": "https://httpbin.org/uuid",
  "JSON": "https://httpbin.org/json",
}
def get_content(title, url):
  resp = requests.get(url)
  print(f"Response for {title}")
  print(resp.json())
for title, url in urls.items():
  get_content(title, url)
  print("-" * 40)

# **Sincronizar datos con Pickle**

In [ ]:
import pickle
from dataclasses import dataclass
@dataclass
class Person:
  first_name: str
  last_name: str
  id: int
  def greet(self):
    print(f'Hola, soy {self.first_name} {self.last_name}'
      f' y mi ID es {self.id}')
people = [
    Person('Obi-Wan', 'Kenobi', 123),
    Person('Anakin', 'Skywalker', 456),
]
# Guardamos los datos en un archivo binario
with open('data.pickle', 'wb') as stream:
  pickle.dump(people, stream)
# Caergamos los datos del archivo
with open('data.pickle', 'rb') as stream:
  peeps = pickle.load(stream)
for person in peeps:
  person.greet()

# **Guardar datos con Shelve**

In [ ]:
import shelve
class Person:
  def __init__(self, name, id):
        self.name = name
        self.id = id
with shelve.open('shelf1.shelve') as db:
    db['obi1'] = Person('Obi-Wan', 123)
    db['ani'] = Person('Anakin', 456)
    db['a_list'] = [2, 3, 5]
    db['delete_me'] = 'we will have to delete this one...'
    print(list(db.keys())) # 'ani', 'delete_me', 'a_list', 'obi1']
    del db['delete_me'] # gone!
    print(list(db.keys())) # ['ani', 'a_list', 'obi1']
    print('delete_me' in db) # False
    print('ani' in db) # True
    a_list = db['a_list']
    a_list.append(7)
    db['a_list'] = a_list
    print(db['a_list']) # [2, 3, 5, 7]

# **Guardando datos en una base de datos**

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (
Column, Integer, String, ForeignKey, create_engine)
from sqlalchemy.orm import relationship

engine = create_engine('sqlite:///:memory:')
Base = declarative_base()
class Person(Base):
    __tablename__ = 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    addresses = relationship(
        'Address',
        back_populates='person',
        order_by='Address.email',
        cascade='all, delete-orphan'
    )
    def __repr__(self):
        return f'{self.name}(id={self.id})'
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)

    email = Column(String)
    person_id = Column(ForeignKey('person.id'))
    person = relationship('Person', back_populates='addresses')
    def __str__(self):
        return self.email
    __repr__ = __str__
Base.metadata.create_all(engine)